In [ ]:
# ============================================================
# Setup do repositório
# ============================================================
# Quando abrimos um notebook pelo GitHub no Google Colab,
# apenas o arquivo .ipynb é carregado.
# Esta célula garante que TODO o repositório esteja disponível.

import os

REPO_URL = "https://github.com/vongrossi/fazendo-um-llm-do-zero.git"
REPO_DIR = "fazendo-um-llm-do-zero"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

os.chdir(REPO_DIR)

print("Diretório atual:", os.getcwd())
print("Conteúdo da raiz:", os.listdir("."))


## Capítulo 01 — O que é um LLM (de verdade)

Este notebook acompanha o Capítulo 01 da série **Fazendo um LLM do Zero**.

🎯 Objetivos deste notebook:
- Entender LLMs como modelos de previsão do próximo token
- Ver como comportamento emerge de regras simples
- Conectar o conceito teórico com um exemplo prático
- Preparar o terreno para tokenização e Transformers


### Instalação de dependências

In [ ]:
# ============================================================
# Instalação de dependências do capítulo
# ============================================================
# No Colab, muitas bibliotecas já vêm instaladas,
# mas usamos este passo para garantir consistência.

!pip -q install -r 01-o-que-e-um-llm/requirements.txt

### Imports e ambiente

In [ ]:
# ============================================================
# Imports básicos
# ============================================================

import sys
import numpy as np
import random

# Permite importar o colab_setup.py do capítulo
sys.path.append("01-o-que-e-um-llm")

from colab_setup import seed_everything

seed_everything(42)

print("Ambiente configurado com seed fixa.")


### Entendendo o Conceito
#### LLMs começam com algo simples

Antes de Transformers, atenção ou bilhões de parâmetros,
um LLM começa resolvendo uma tarefa básica:

> **Dado um contexto, prever o próximo token mais provável.**

Neste notebook, vamos construir um modelo extremamente simples
que faz exatamente isso — sem deep learning ainda.

A ideia é mostrar que **comportamento interessante pode emergir**
mesmo de regras estatísticas simples.


In [ ]:
# ============================================================
# Dataset simples de frases
# ============================================================
# Em LLMs reais, usamos bilhões de tokens.
# Aqui usamos um conjunto pequeno apenas para entendimento.

sentences = [
    "o gato subiu no telhado",
    "o cachorro subiu no sofá",
    "o gato dormiu no sofá",
    "o cachorro dormiu no tapete",
    "o gato pulou no muro",
]

print("Dataset de exemplo:")
for s in sentences:
    print("-", s)


### Tokenização simples (por palavra)

In [ ]:
# ============================================================
# Tokenização simples
# ============================================================
# Aqui, cada palavra será tratada como um token.
# Isso NÃO é como LLMs reais fazem, mas ajuda a entender a ideia.

tokenized_sentences = [s.split() for s in sentences]

print("Frases tokenizadas:")
for ts in tokenized_sentences:
    print(ts)


### Construindo estatísticas de próximo token

In [ ]:
# ============================================================
# Contando transições de tokens
# ============================================================
# Vamos contar:
# Dado um token atual, quais tokens costumam vir depois?

from collections import defaultdict

next_token_counts = defaultdict(lambda: defaultdict(int))

for sentence in tokenized_sentences:
    for i in range(len(sentence) - 1):
        current_token = sentence[i]
        next_token = sentence[i + 1]
        next_token_counts[current_token][next_token] += 1

print("Exemplo de transições aprendidas:")
for token, transitions in next_token_counts.items():
    print(f"\n'{token}' → {dict(transitions)}")


### Convertendo contagens em probabilidades

In [ ]:
# ============================================================
# Convertendo contagens em probabilidades
# ============================================================

next_token_probs = {}

for token, transitions in next_token_counts.items():
    total = sum(transitions.values())
    probs = {t: count / total for t, count in transitions.items()}
    next_token_probs[token] = probs

print("Probabilidades aprendidas:")
for token, probs in next_token_probs.items():
    print(f"\nApós '{token}':")
    for t, p in probs.items():
        print(f"  {t}: {p:.2f}")


### Gerando texto (previsão do próximo token)

In [ ]:
# ============================================================
# Função simples de geração de texto
# ============================================================

def generate_text(start_token, max_steps=10):
    current_token = start_token
    generated = [current_token]

    for _ in range(max_steps):
        if current_token not in next_token_probs:
            break

        tokens = list(next_token_probs[current_token].keys())
        probabilities = list(next_token_probs[current_token].values())

        # Escolhe o próximo token com base na distribuição
        current_token = random.choices(tokens, probabilities)[0]
        generated.append(current_token)

    return " ".join(generated)


### Testando o “modelo”

In [ ]:
# ============================================================
# Testes de geração
# ============================================================

for start in ["o", "gato", "cachorro"]:
    print(f"\nComeçando com '{start}':")
    print(generate_text(start))


### O que acabamos de construir?

Este modelo extremamente simples:

- não entende linguagem
- não tem consciência
- não sabe o significado das palavras

E ainda assim:
- aprende padrões
- respeita contexto local
- gera texto plausível

Isso acontece porque ele aprendeu **probabilidades condicionais**.

LLMs modernos fazem exatamente isso,
mas com:
- tokenização muito mais sofisticada
- contextos muito maiores
- arquiteturas como Transformers
- bilhões de parâmetros

O princípio fundamental, porém, é o mesmo.


### Conexão com LLMs reais

O que você viu aqui é uma versão microscópica da ideia central dos LLMs.

Nos próximos capítulos, vamos:
- melhorar a tokenização
- aumentar o contexto
- substituir estatística simples por redes neurais
- introduzir atenção e Transformers

Mas sempre mantendo a mesma pergunta no centro:

> “Dado tudo que veio antes, qual é o próximo token mais provável?”


### 🧾 Glossário Rápido

**Token**  
Unidade básica de texto processada pelo modelo.

**Previsão do próximo token**  
Tarefa central dos modelos de linguagem: estimar o token mais provável dado um contexto.

**Probabilidade condicional**  
Probabilidade de um evento acontecer dado que outro já ocorreu.

**Modelo de linguagem**  
Modelo que atribui probabilidades a sequências de tokens.
